# VQE for upper spins

Author: Javier Norambuena Leiva — Posted: . Last updated: .

Here we discust how to implement the variational quantum eigensolver for spin systems (chain , rings and more complex structures) when this one is different of one half.

In [1]:
import math
import itertools
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import scipy.linalg as la
import scipy as sc

## The main idea

The variational quantum eigensolver (VQE) is a central computational method in the NISQ era for its robustness against noise. The majority of tutorials that involve VQE with a Heisenberg model only consider the base case, which is spin one-half.

This is a limitation when one tries to study spin systems that have upper spins, for example, the single molecular magnet 3Ni is studied with a Heisenberg model, but each site has a spin of one. Here, we propose an approach to studying this kind of system when the spin of the model is different from one-half.

To perform this, we need to focus on two things: the expected value formula and the basis representation. Let's begin with the latter. Let's consider a spin-1. We know that this spin is represented as a three-level system. Therefore, an arbitrary state is written as:

$$\ket{\psi} = \alpha\ket{0} + \beta\ket{1} + \gamma\ket{2}$$

The key idea is to try to rewrite these three states into an n-qubit set. Thus, we need to find the minimum number of qubits that allow us to contain our set of states in the basis of the n-qubit. For our example case, we need two qubits. So, we can encode it like this:

$$\ket{0} \longrightarrow \ket{00}$$
$$\ket{1} \longrightarrow \ket{01}$$
$$\ket{2} \longrightarrow \ket{10}$$

This leaves us with a state in the qubit that doesn't have any meaning. Thus, we need to tell the method to try not to choose the left state. A naive idea could be to add a penalty term if some samples go to the left state.

Let's leave this for a moment and move to the expected value formula. Let's remember the formula for the expected value of the Z Pauli operator of spin one-half:

$$ \bra{\psi}Z\ket{\psi} = \bra{\psi}( \ket{0}\bra{0} - \ket{1}\bra{1} )\ket{\psi} = |\bra{0}\ket{\psi}|^2 - |\bra{1}\ket{\psi}|^2$$

As we can see, the expected value is the difference of the projection of $\ket{\psi}$ into the basis states. This approach is the same for upper spins. If we change to the Z Pauli operator of spin one, the formula changes a little bit:

$$ \bra{\psi}Z\ket{\psi} = \bra{\psi}( \ket{0}\bra{0} - \ket{2}\bra{2} )\ket{\psi} = |\bra{0}\ket{\psi}|^2 - |\bra{2}\ket{\psi}|^2$$

And as we defined above, this probability can be written in the 2-qubit basis, giving us the following result:
$$ |\bra{00}\ket{\psi}|^2 - |\bra{10}\ket{\psi}|^2$$

The before formula was for one qutrit, but what happen with interaction between qutrit. Two qutrit create a space of 9 basis states, so we need 4-qubits to represent it, but how we codify this,  

This idea for spin one can be used for every spin with the condition of knowing the states that compose the Z Pauli operator (in other words, having the matrix).

Here, you can find the explicit definition of the Pauli matrices for spin 5/2 (https://easyspin.org/easyspin/documentation/spinoperators.html).

## System definition

Let consider a ring of three qudits of spin 1 and 1/5 that is represented using a Heisenberg model without external field, to represent the hamiltonian, we will use a list of Pauli string representation.

In [ ]:
nr_qudits = 3
list_spins = [1.0, 1.5, 2.0]


## Creation of Ansatz Circuit

In [ ]:
def single_rotation(phi_params, qubits, spin):
        phi_params = np.array(phi_params).T
        correction = math.ceil( (int( 2*spin+1 ))/2  )
        for i in range( 0, qubits):
            for j in range(correction):
                qml.RZ(phi_params[i][0], wires=[correction*i+j])
                qml.RY(phi_params[i][1], wires=[correction*i+j])
                qml.RX(phi_params[i][2], wires=[correction*i+j])
        return

def non_local_gates(phi_params, qubits, spin):
        correction = math.ceil( (int( 2*spin+1 ))/2  )
        for i in range(0, qubits-1):
            for j in range(correction):
                qml.CRX(phi_params[i], [correction*i+j, correction*(i+1)+j])
        return

In [ ]:
def quantum_circuit(qubits, spin, repetition, rotation_params, coupling_params, wire, sample=None):
        correction = math.ceil( (int( 2*spin+1 ))/2  )
        qml.BasisState(sample, wires=range(correction*qubits))
        for i in range(0, repetition):
            single_rotation(rotation_params[i], qubits, spin)
            non_local_gates(coupling_params[i], qubits, spin)
        index_wire = []
        for w in wire:
            for i in range(correction):
                index_wire.append( correction*w + i)
        return qml.counts(wires=index_wire)

## Cost function

In [ ]:
def cost_function_VQE(theta: list) -> float:
        ansatz_params_1 = theta[0 :number_nonlocal]
        ansatz_params_2 = theta[number_nonlocal: ]
        coupling = np.split(ansatz_params_1, self.repetition)
        split = np.split(ansatz_params_2, self.repetition)
        rotation = []
        for s in split:
            rotation.append(np.split(s, 3))

        if self.spin != 0.5:
            result= 0.0
            for i, term in enumerate(self.hamiltonian_index):
                result_term = self.node( qubits= self.number_qubits, spin=self.spin, rotation_params = rotation, coupling_params = coupling, wire = term, sample=state )
                for _, dict_term in enumerate( conts_spin[ str(self.spin) ]["2"] ):
                    if dict_term in result_term:
                        exchange = self.hamiltonian_object[i][1]
                        prob = result_term[dict_term]/3000.0
                        const_state = conts_spin[ str(self.spin) ]["2"][dict_term]
                        result += exchange*prob*const_state
        else:
            result = self.node( qubits= self.number_qubits, spin=self.spin, rotation_params = rotation, 
                coupling_params = coupling, wire = range(self.number_qubits), sample=state, system_object= self.hamiltonian_object )
            print(result)
        return result

In [ ]:
hf = qml.qchem.hf_state(electrons*correction, self.number_qubits*correction)
general_cost_function = lambda theta: self.cost_function_VQE(theta, hf)
xs = sc.optimize.minimize(general_cost_function, theta, method=self.optimization_method[0],
    options=self.optimization_alg_params[0])['x']

## Numerical test